# Chapter 1: Characterizing Seasonal Rainfall Variability

Rainfall in many dryland ecosystems exhibits pronounced variability from year to year. In this excercise, we will investigate the nature of rainfall at some locations in Kenya and Zambia and use these data to draw inference into the nature of the rainfall process. Finally, we will use the data to develop an empirical model of rainfall that we could use to simulate the rainfall process across many realizations of discrete growing seasons. 

## Chapter Objectives

* Analyze daily rainfall at monthly, seasonal, and yearly scales
* Examine the nature of the daily rainfall process
* Plot key results and compare findings across stations and over time.

## Reading in the data

All the data for our excercise are stored as `.csv` files. These files are very easy to import into pandas.

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("../data/archers_post_rainfall.csv")
df.head()

,Rainfall (mm),Year,Month,Day
0,0,1958.0,1.0,1.0
1,0,1958.0,1.0,2.0
2,0,1958.0,1.0,3.0
3,0,1958.0,1.0,4.0
4,0,1958.0,1.0,5.0


In [7]:
print max(df['Year'])

2003.0